In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

X = pd.read_csv("../data/train_values.csv")
y = pd.read_csv("../data/train_labels.csv")
y = y.set_index("building_id")
X = X.set_index("building_id")

In [ ]:
from dmgpred.cleaning import clean

X = clean(X)[0]

In [ ]:
X.info()

In [ ]:
# invert the encoding of the one-hot encoded columns

has_superstructure = [col for col in X.columns if col.startswith("has_superstructure")]
has_secondary_use = [col for col in X.columns if col.startswith("has_secondary_use")]


# pd.from_dummies(X[has_secondary_use]).head()

In [ ]:
prefix = "has_superstructure"
prefix_len = 2
sep = "_"
cols = [col for col in X.columns if col.startswith(prefix)]


def reverse_multi_hot_encoding(row, cols: list[str], sep="_", prefix_len=2):
    """Reverse the multi-hot encoding of the given cols."""
    suffixes = []
    for col in cols:
        suffix = col.split(sep)[prefix_len:]
        suffix = sep.join(suffix)
        if row[col] == 1:
            suffixes.append(suffix)
    return suffixes


X = X.assign(
    has_superstructure=X.apply(
        reverse_multi_hot_encoding, axis=1, args=(cols, sep, prefix_len)
    )
)

In [ ]:
import category_encoders as ce

In [ ]:
encoder = ce.HashingEncoder(cols="has_superstructure", n_components=4)
encoder.fit_transform(X)

In [ ]:
ce.TargetEncoder(cols=X.select_dtypes(include="category").columns).fit_transform(X, y)